In [2]:
import os
import nibabel as nib
import numpy as np
import imageio

# Define the main paths
dataset_folder = r"C:\Users\pujau\OneDrive\Documents\3d\kits19\data"
output_folder = r"C:\Users\pujau\OneDrive\Documents\thesis\kits19\data\all_data"

# Create output directory if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Function to normalize and convert the slices to 8-bit integers
def normalize_and_convert(slice_2d):
    # Normalize the slice to range [0, 255]
    slice_2d = 255 * (slice_2d - np.min(slice_2d)) / (np.max(slice_2d) - np.min(slice_2d))
    return slice_2d.astype(np.uint8)

# Function to save 2D slices
def save_slices(data, case_id, slice_type):
    case_output_folder = os.path.join(output_folder, case_id, slice_type)
    os.makedirs(case_output_folder, exist_ok=True)
    
    for i in range(data.shape[0]):
        slice_2d = data[i, :, :]
        slice_2d = normalize_and_convert(slice_2d)
        output_path = os.path.join(case_output_folder, f"{slice_type}_slice_{i:03d}.png")
        imageio.imwrite(output_path, slice_2d)

# Function to save 3-channel segmentation slices
def save_3_channel_slices(segmentation_data, case_id):
    case_output_folder = os.path.join(output_folder, case_id, "segmentation")
    os.makedirs(case_output_folder, exist_ok=True)
    
    num_classes = 3  # Define the number of classes (background, kidney, tumor)
    for i in range(segmentation_data.shape[0]):
        slice_2d = segmentation_data[i, :, :]
        
        # Create an empty 3-channel image
        multi_channel_slice = np.zeros((slice_2d.shape[0], slice_2d.shape[1], num_classes), dtype=np.uint8)
        
        # Assign each class to a specific channel
        for c in range(num_classes):
            class_slice = (slice_2d == c).astype(np.uint8)  
            multi_channel_slice[:, :, c] = class_slice
        
        output_path = os.path.join(case_output_folder, f"segmentation_slice_{i:03d}.png")
        imageio.imwrite(output_path, multi_channel_slice)

# Main loop over each case
for case_id in os.listdir(dataset_folder):
    case_folder = os.path.join(dataset_folder, case_id)
    
    if os.path.isdir(case_folder):
        imaging_path = os.path.join(case_folder, "imaging.nii.gz")
        segmentation_path = os.path.join(case_folder, "segmentation.nii.gz")
        
        if os.path.exists(imaging_path) and os.path.exists(segmentation_path):
            # Load the NIfTI files
            imaging = nib.load(imaging_path)
            segmentation = nib.load(segmentation_path)
            
            # Get the data arrays
            imaging_data = imaging.get_fdata()
            segmentation_data = segmentation.get_fdata()
            
            # Save 2D slices of imaging data
            save_slices(imaging_data, case_id, "imaging")
            
            # Save 3-channel segmentation slices
            save_3_channel_slices(segmentation_data, case_id)



In [3]:
from PIL import Image
import numpy as np

# Path to your segmentation image
image_path = r"C:\Users\pujau\OneDrive\Documents\3d\kits19\all_data\case_00000\segmentation\segmentation_slice_000.png"

# Load the image
image = Image.open(image_path)

# Convert image to NumPy array
image_array = np.array(image)

# Print the shape of the image
print("Image shape:", image_array.shape)


Image shape: (512, 512, 3)
